### Invoke the Multi-agent
---

This notebook contains simple logic to invoke the Multi agent that is created in your account. This notebook requires the following information, assuming that the agentic system is already created in your account:

1. **Multi-agent, and sub agents IDs and Alias IDs**: This information can be found on the Bedrock console for the respective agents.

1. **Names of the agents**.

In [ ]:
import os
import sys
import uuid
import json
import time
import boto3
import logging
from dotenv import load_dotenv

# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)
from globals import *

In [ ]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# configure the sts client, the boto3 session and other variables
sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

### Load the config file
--- 

Load the config file that contains information on the supervisor routing logic.

In [ ]:
import sys
import pandas as pd
sys.path.insert(0, ".")
sys.path.insert(1, "..")
# Import utility functions and helper functions for agents
from utils.utils import *

In [ ]:
# Get the absolute path to the config file
# This config file contains data about the directory paths, the API specs that
# are used to generate the code, and the agent foundation models that are used to generate the code.
BASE_DIR = os.path.abspath(sys.path[1])
CONFIG_FPATH = os.path.join(BASE_DIR, CONFIG_FNAME)
config_data = load_config(CONFIG_FPATH)
logger.info(f"Loaded config from local file system: {json.dumps(config_data, indent=2)}")

In [ ]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
agents = AgentsForAmazonBedrock()

#### Extract the Multi agent information

In [ ]:
# Extract agent IDs and alias IDs from config_data
home_networking_id = '<your-home-networking-assistant-id>'
home_networking_alias_id = '<your-home-networking-assistant-alias-id>'
doorbell_id = '<your-doorbell-configuration-assistant-id>'
doorbell_alias_id = '<your-doorbell-configuration-assistant-alias-id>'
multi_agent_id = '<your-multi-agent-id>'

# Create the multi_agent_names dictionary
multi_agent_names = {
    f"{home_networking_id}/{home_networking_alias_id}": '<your-home-assistant-agent-name>',
    f"{doorbell_id}/{doorbell_alias_id}": '<your-doorbell-configuration-agent-name>'
}

In [ ]:
multi_agent_names

#### Invoke the multi agent

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    """Is my living room camera online? My devideId is madhur2039.""", 
    multi_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)